In [1]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "4"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3052,application_1732639283265_3010,pyspark,idle,Link,Link,None,
3082,application_1732639283265_3038,pyspark,idle,Link,Link,None,
3084,application_1732639283265_3040,pyspark,idle,Link,Link,None,
3087,application_1732639283265_3043,pyspark,idle,Link,Link,None,
3089,application_1732639283265_3045,pyspark,idle,Link,Link,None,
3100,application_1732639283265_3056,pyspark,idle,Link,Link,None,
3102,application_1732639283265_3058,pyspark,idle,Link,Link,None,
3112,application_1732639283265_3068,pyspark,idle,Link,Link,None,
3116,application_1732639283265_3072,pyspark,idle,Link,Link,None,
3125,application_1732639283265_3081,pyspark,idle,Link,Link,None,


In [2]:
# Spark Dataframe code
from pyspark.sql import SparkSession,Row
from pyspark.sql.functions import col,udf,when
from pyspark.sql.types import StringType
import time
results = []


spark = SparkSession.builder.appName("Query 1 DF").getOrCreate()
conf = spark.sparkContext.getConf()

# Print relevant executor settings
print("Executor Instances:", conf.get("spark.executor.instances"))
print("Executor Memory:", conf.get("spark.executor.memory"))
print("Executor Cores:", conf.get("spark.executor.cores"))

spark.catalog.clearCache()
csv_files = [
    "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv",
    "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
]
#read the dataset and perform filtering
crime_data_df = spark.read.csv(csv_files, header=True, inferSchema=True)
start_time = time.time()
selected_columns_df = crime_data_df.select(col("Vict Age").cast("int"),"Crm Cd Desc")
filtered_df = selected_columns_df.filter(
    (col("Vict Age") > 0) & 
    (col("Crm Cd Desc").contains("AGGRAVATED ASSAULT"))
)
#categorize by age
filtered_df.withColumn(
    "group",
    when(col("Vict Age") < 18, "Children")
    .when(col("Vict Age") < 25, "Young Adults")
    .when(col("Vict Age") < 65, "Adults")
    .otherwise("Elderly")
).groupBy("group").count().show()
end_time = time.time()
results.append(Row(
            Data_Abstractions="Dataframe",
            Execution_Time=end_time - start_time
        ))
results_df = spark.createDataFrame(results)
# Save the DataFrame to S3 as JSON, we save the results because we restart the kernel in every execution
results_df.write.mode("append").json("s3://groups-bucket-dblab-905418150721/group45/q1/results.json")


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3276,application_1732639283265_3232,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Executor Instances: 4
Executor Memory: 4743M
Executor Cores: 2
+------------+------+
|       group| count|
+------------+------+
|      Adults|121093|
|    Children| 10830|
|     Elderly|  5985|
|Young Adults| 33605|
+------------+------+

In [3]:
# execute this after you run RDD file first and the file after that. Remember to restart the kernel
spark.read.json("s3://groups-bucket-dblab-905418150721/group45/q1/results.json").show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-----------------+
|Data_Abstractions|   Execution_Time|
+-----------------+-----------------+
|        Dataframe|5.989339351654053|
|              RDD|7.777620315551758|
+-----------------+-----------------+